# Predicting Crime in US Communites
## Team member: Bibata Rabba Idi, Fatima Javid, JianHui (Jake) Li 

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns

In [3]:
# Load data
df = pd.read_csv('crimedata.csv')
df.head()

,communityName,state,countyCode,communityCode,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,...,burglaries,burglPerPop,larcenies,larcPerPop,autoTheft,autoTheftPerPop,arsons,arsonsPerPop,ViolentCrimesPerPop,nonViolPerPop
0,BerkeleyHeightstownship,NJ,39.0,5320.0,11980,3.10,1.37,91.78,6.50,1.88,...,14.0,114.85,138.0,1132.08,16.0,131.26,2.0,16.41,41.02,1394.59
1,Marpletownship,PA,45.0,47616.0,23123,2.82,0.80,95.57,3.44,0.85,...,57.0,242.37,376.0,1598.78,26.0,110.55,1.0,4.25,127.56,1955.95
2,Tigardcity,OR,NaN,NaN,29344,2.43,0.74,94.33,3.43,2.35,...,274.0,758.14,1797.0,4972.19,136.0,376.30,22.0,60.87,218.59,6167.51
3,Gloversvillecity,NY,35.0,29443.0,16656,2.40,1.70,97.35,0.50,0.70,...,225.0,1301.78,716.0,4142.56,47.0,271.93,NaN,NaN,306.64,NaN
4,Bemidjicity,MN,7.0,5068.0,11245,2.76,0.53,89.16,1.17,0.52,...,91.0,728.93,1060.0,8490.87,91.0,728.93,5.0,40.05,NaN,9988.79


### Variable Definition
communityName:

state:

countyCode:

communityCode:

population:

householdsize:

racepctblack:

racePctWhite:

racePctAsian:

racePctHisp:

agePct12t21:

agePct12t29:

agePct16t24:

agePct65up:

numbUrban:

pctUrban:

medIncome:

pctWWage:

pctWFarmSelf:

pctWInvInc:

pctWSocSec:

pctWPubAsst:

pctWRetire:

medFamInc:

perCapInc:

whitePerCap:

blackPerCap:

indianPerCap:

AsianPerCap:

OtherPerCap:

HispPerCap:

NumUnderPov:

PctPopUnderPov:

PctLess9thGrade:

PctNotHSGrad:

PctBSorMore:

PctUnemployed:

PctEmploy:

PctEmplManu:

PctEmplProfServ:

PctOccupManu:

PctOccupMgmtProf:

MalePctDivorce:

MalePctNevMarr:

FemalePctDiv:

TotalPctDiv:

PersPerFam:

PctFam2Par:

PctKids2Par:

PctYoungKids2Par:

PctTeen2Par:

PctWorkMomYoungKids:

PctWorkMom:

NumKidsBornNeverMar:

PctKidsBornNeverMar:

NumImmig:

PctImmigRecent:

PctImmigRec5:

PctImmigRec8:

PctImmigRec10:

PctRecentImmig:

PctRecImmig5:

PctRecImmig8:

PctRecImmig10:

PctSpeakEnglOnly:

PctNotSpeakEnglWell:

PctLargHouseFam:

PctLargHouseOccup:

PersPerOccupHous:

PersPerOwnOccHous:

PersPerRentOccHous:

PctPersOwnOccup:

PctPersDenseHous:

PctHousLess3BR:

MedNumBR:

HousVacant:

PctHousOccup:

PctHousOwnOcc:

PctVacantBoarded:

PctVacMore6Mos:

MedYrHousBuilt:

PctHousNoPhone:

PctWOFullPlumb:

OwnOccLowQuart:

OwnOccMedVal:

OwnOccHiQuart:

OwnOccQrange:

RentLowQ:

RentMedian:

RentHighQ:

RentQrange:

MedRent:

MedRentPctHousInc:

MedOwnCostPctInc:

MedOwnCostPctIncNoMtg:

NumInShelters:

NumStreet:

PctForeignBorn:

PctBornSameState:

PctSameHouse85:

PctSameCity85:

PctSameState85:

LemasSwornFT:

LemasSwFTPerPop:

LemasSwFTFieldOps:

LemasSwFTFieldPerPop:

LemasTotalReq:

LemasTotReqPerPop:

PolicReqPerOffic:

PolicPerPop:

RacialMatchCommPol:

PctPolicWhite:

PctPolicBlack:

PctPolicHisp:

PctPolicAsian:

PctPolicMinor:

OfficAssgnDrugUnits:

NumKindsDrugsSeiz:

PolicAveOTWorked:

LandArea:

PopDens:

PctUsePubTrans:

PolicCars:

PolicOperBudg:

LemasPctPolicOnPatr:

LemasGangUnitDeploy:

LemasPctOfficDrugUn:

PolicBudgPerPop:

murders:

murdPerPop:

rapes:

rapesPerPop:

robberies:

robbbPerPop:

assaults:

assaultPerPop:

burglaries:

burglPerPop:

larcenies:

larcPerPop:

autoTheft:

autoTheftPerPop:

arsons:

arsonsPerPop:

ViolentCrimesPerPop:

nonViolPerPop:
